In [1]:
import os
os.getcwd()
from pathlib import Path

DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)

DATA_DIR.resolve()



PosixPath('/Users/himaj/Documents/Applied_Science/ranking/data')

In [2]:
%pip install rank-bm25



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /Users/himaj/Downloads/ml_assign/code/ml_env/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import urllib.request

url = "https://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz"
out_path = DATA_DIR / "reviews_Electronics_5.json.gz"

if not out_path.exists():
    urllib.request.urlretrieve(url, out_path)

out_path


PosixPath('../data/reviews_Electronics_5.json.gz')

In [4]:
import gzip, json
import pandas as pd

records = []

with gzip.open(DATA_DIR / "reviews_Electronics_5.json.gz", "rt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))
        if i >= 20000:   # limit for now
            break

df = pd.DataFrame(records)
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [5]:

rank_df = df[["summary", "reviewText", "overall", "asin"]].dropna()


rank_df["relevance"] = rank_df["overall"].astype(int)


rank_df = rank_df.rename(columns={
    "summary": "query",
    "reviewText": "doc"
})

rank_df.head()


,query,doc,overall,asin,relevance
0,Gotta have GPS!,We got this GPS for my husband who is an (OTR)...,5.0,0528881469,5
1,Very Disappointed,"I'm a professional OTR truck driver, and I bou...",1.0,0528881469,1
2,1st impression,"Well, what can I say. I've had this unit in m...",3.0,0528881469,3
3,"Great grafics, POOR GPS","Not going to write a long review, even thought...",2.0,0528881469,2
4,"Major issues, only excuses for support",I've had mine for a year and here's what we go...,1.0,0528881469,1


In [6]:

grouped = rank_df.groupby("query")
rank_df = grouped.filter(lambda x: len(x) >= 5)

print("Total rows:", len(rank_df))
print("Unique queries:", rank_df["query"].nunique())


Total rows: 2101
Unique queries: 181


In [7]:

rank_df = rank_df.copy()
rank_df["query_id"] = rank_df["query"].astype("category").cat.codes


rank_df["doc_id"] = rank_df.index

rank_df[["query_id", "query", "doc_id", "relevance", "asin"]].head(10)


,query_id,query,doc_id,relevance,asin
10,6,As expected,10,4,0594481813
12,127,Works Great,12,5,0594481813
13,89,It Works,13,3,0594481813
19,108,Perfect,19,5,0972683275
25,125,Works,25,4,0972683275
31,165,great,31,5,0972683275
39,92,It works!,39,4,0972683275
40,116,Very Pleased,40,5,0972683275
42,151,Works well,42,4,0972683275
45,167,great product,45,5,0972683275


In [8]:

rank_df = rank_df.copy()
rank_df["query_id"] = rank_df["query"].astype("category").cat.codes

rank_df["doc_id"] = rank_df.index

rank_df[["query_id", "query", "doc_id", "relevance", "asin"]].head(10)


,query_id,query,doc_id,relevance,asin
10,6,As expected,10,4,0594481813
12,127,Works Great,12,5,0594481813
13,89,It Works,13,3,0594481813
19,108,Perfect,19,5,0972683275
25,125,Works,25,4,0972683275
31,165,great,31,5,0972683275
39,92,It works!,39,4,0972683275
40,116,Very Pleased,40,5,0972683275
42,151,Works well,42,4,0972683275
45,167,great product,45,5,0972683275


In [9]:
import numpy as np


qids = rank_df["query_id"].unique()
np.random.seed(42)
np.random.shuffle(qids)

split = int(0.8 * len(qids))
train_qids = set(qids[:split])
test_qids  = set(qids[split:])

train_df = rank_df[rank_df["query_id"].isin(train_qids)].copy()
test_df  = rank_df[rank_df["query_id"].isin(test_qids)].copy()

print("Train rows:", len(train_df), " Train queries:", train_df["query_id"].nunique())
print("Test rows :", len(test_df),  " Test queries :", test_df["query_id"].nunique())


Train rows: 1708  Train queries: 144
Test rows : 393  Test queries : 37


In [10]:
import re

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.split()
from rank_bm25 import BM25Okapi
from collections import defaultdict
def bm25_rank(query, docs):
    tokenized_docs = [tokenize(d) for d in docs]
    bm25 = BM25Okapi(tokenized_docs)
    query_tokens = tokenize(query)
    scores = bm25.get_scores(query_tokens)
    return scores
import numpy as np

def dcg_at_k(relevances, k):
    relevances = np.array(relevances)[:k]
    return np.sum((2**relevances - 1) / np.log2(np.arange(2, len(relevances) + 2)))

def ndcg_at_k(relevances, k):
    ideal = sorted(relevances, reverse=True)
    ideal_dcg = dcg_at_k(ideal, k)
    if ideal_dcg == 0:
        return 0.0
    return dcg_at_k(relevances, k) / ideal_dcg
def evaluate_bm25(df, k=10):
    ndcgs = []

    for qid, group in df.groupby("query_id"):
        query_text = group["query"].iloc[0]
        docs = group["doc"].tolist()
        relevances = group["relevance"].tolist()

        scores = bm25_rank(query_text, docs)
        ranked_indices = np.argsort(scores)[::-1]

        ranked_relevances = [relevances[i] for i in ranked_indices]
        ndcgs.append(ndcg_at_k(ranked_relevances, k))

    return np.mean(ndcgs)
bm25_ndcg = evaluate_bm25(test_df, k=10)
bm25_ndcg


np.float64(0.942973213787023)

In [11]:
import numpy as np
import pandas as pd

def make_pairs(df, max_pairs_per_query=50, seed=42):
    rng = np.random.default_rng(seed)
    pairs = []
    
    for qid, g in df.groupby("query_id"):
        g = g.reset_index(drop=True)
        
        pos = g[g["relevance"] >= 4]
        neg = g[g["relevance"] <= 2]
        if len(pos) == 0 or len(neg) == 0:
            continue
        
        # sample pairs
        for _ in range(min(max_pairs_per_query, len(pos)*len(neg))):
            p = pos.iloc[rng.integers(0, len(pos))]
            n = neg.iloc[rng.integers(0, len(neg))]
            pairs.append({
                "query": p["query"],
                "doc_pos": p["doc"],
                "doc_neg": n["doc"],
                "qid": qid,
                "rel_pos": int(p["relevance"]),
                "rel_neg": int(n["relevance"]),
            })
    return pd.DataFrame(pairs)

train_pairs = make_pairs(train_df, max_pairs_per_query=30)
test_pairs  = make_pairs(test_df,  max_pairs_per_query=30)

print("Train pairs:", len(train_pairs))
print("Test pairs :", len(test_pairs))
train_pairs.head()


Train pairs: 55
Test pairs : 24


,query,doc_pos,doc_neg,qid,rel_pos,rel_neg
0,Filter,Nice filter great for sky and water shots. I l...,when I used it I found it softened my sharpnes...,28,4,2
1,Filter,Wanted a filter to protect the glass on a new ...,when I used it I found it softened my sharpnes...,28,5,2
2,Filter,It does nothing but protect the camera lens fr...,when I used it I found it softened my sharpnes...,28,5,2
3,Filter,No to much to say. Its a camera filter and it ...,when I used it I found it softened my sharpnes...,28,4,2
4,Filter,No to much to say. Its a camera filter and it ...,when I used it I found it softened my sharpnes...,28,4,2


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


def build_examples(pairs_df):
    X = []
    y = []
    for _, row in pairs_df.iterrows():
        # positive example
        X.append(row["query"] + " [SEP] " + row["doc_pos"])
        y.append(1)
        # negative example (swap)
        X.append(row["query"] + " [SEP] " + row["doc_neg"])
        y.append(0)
    return X, np.array(y)

X_train, y_train = build_examples(train_pairs)
X_test,  y_test  = build_examples(test_pairs)

model = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1,2))),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
pred = model.predict(X_test)

print("Pairwise classification accuracy:", accuracy_score(y_test, pred))


Pairwise classification accuracy: 0.5


In [13]:
def model_score(query, docs, model):
    texts = [query + " [SEP] " + d for d in docs]
   
    return model.predict_proba(texts)[:, 1]
def evaluate_model_ranking(df, model, k=10):
    ndcgs = []

    for qid, group in df.groupby("query_id"):
        query_text = group["query"].iloc[0]
        docs = group["doc"].tolist()
        relevances = group["relevance"].tolist()

        scores = model_score(query_text, docs, model)
        ranked_indices = np.argsort(scores)[::-1]

        ranked_relevances = [relevances[i] for i in ranked_indices]
        ndcgs.append(ndcg_at_k(ranked_relevances, k))

    return np.mean(ndcgs)
ltr_ndcg = evaluate_model_ranking(test_df, model, k=10)
ltr_ndcg


np.float64(0.9405299448103803)

In [14]:
import pandas as pd

results = pd.DataFrame({
    "Model": ["BM25", "TF-IDF + Logistic (Pairwise LTR)"],
    "NDCG@10": [bm25_ndcg, ltr_ndcg]
})

results


,Model,NDCG@10
0,BM25,0.942973
1,TF-IDF + Logistic (Pairwise LTR),0.940530
